##### Initialize

In [2]:
import sys
sys.path.append('../')

from msal import ConfidentialClientApplication
import pandas as pd
import os
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

import logging

def_credential = DefaultAzureCredential()  

## connect to Azure Key Vault securely
keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"
kv_client = SecretClient(vault_url=KVUri, credential=def_credential)

## Secret Keys for Azure App Registration
tenant_id = kv_client.get_secret('azure-tenant-id').value
client_id = kv_client.get_secret('idgov-app-client-id').value
client_secret = kv_client.get_secret('idgov-app-client-secret').value

## Secret Keys for Bolddesk (Nera Care)
# bd_api_key = kv_client.get_secret ('bolddesk-nera-care-api-key').value
# bd_base_url = kv_client.get_secret('bolddesk-nera-care-api-base-url').value
## Secret Keys for Bolddesk (Nera IT)
bd_api_key = kv_client.get_secret ('bolddesk-nera-it-api-key').value
bd_base_url = kv_client.get_secret('bolddesk-nera-it-api-base-url').value

In [3]:
from module.bolddesk import Bolddesk
from module.warehouse import Warehouse

## Initialize Bolddesk and Warehouse
bd = Bolddesk(bd_base_url, bd_api_key)
wh = Warehouse(
        server=os.environ["DB_SERVER"],
        database=os.environ["DB_NAME"],
        credential=def_credential
)

#### Tickets

In [7]:
df = bd.list_tickets()

date_cols = ['cf_last_date_of_service', 'cf_last_date_of_work', 'cf_last_day_of_retention', 'resolutionDue', 'createdOn', 'closedOn', 'responseDue', 'lastRepliedOn', 'lastUpdatedOn', 'lastStatusChangedOn',]
for col in date_cols:
    df[col] = pd.to_datetime(df[col]).dt.tz_localize(None)

table_name = 'bd_helpdesk_tickets'
wh.erase(table_name)
wh.append(table_name, df)